In [1]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, Conv1D, TimeDistributed, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import backend as K
import time

In [2]:
# 載入資料集
def load_gesture_data(directory):
    gesture_data = []
    labels = []
    
    categories = []
    for category in os.listdir(directory):
        category_path = os.path.join(directory, category)
        if os.path.isdir(category_path):
            categories.append(category)
    
    category_files = {}
    for category in categories:
        category_path = os.path.join(directory, category)
        files = []
        for file in os.listdir(category_path):
            if file.endswith('.npy'):
                files.append(file)
        category_files[category] = files
    
    max_files = 0
    for files in category_files.values():
        if len(files) > max_files:
            max_files = len(files)
    
    for i in range(max_files):
        for category in categories:
            files = category_files[category]
            
            if i < len(files):
                file_name = files[i]
                file_path = os.path.join(category, file_name)
                file_path_full = os.path.join(directory, file_path)
                print(f"Loading {file_path_full}")
                data = np.load(file_path_full)
                gesture_data.append(data)
                labels.append(category)  
    
    return np.array(gesture_data), labels


In [ ]:
# 載入資料集
gesture_data_train, gesture_labels_train = load_gesture_data('train_data/train')
gesture_data_test, gesture_labels_test = load_gesture_data('train_data/test')
gesture_data_val, gesture_labels_val = load_gesture_data('train_data/validation')

print("Original Gesture data train shape:", np.shape(gesture_data_train))
print("Original Gesture labels train shape:", np.shape(gesture_labels_train))

x_train_gesture = np.reshape(gesture_data_train, (-1, 2, 32, 32, 35))
x_test_gesture = np.reshape(gesture_data_test, (-1, 2, 32, 32, 35))
x_val_gesture = np.reshape(gesture_data_val, (-1, 2, 32, 32, 35))

x_train_gesture = np.transpose(x_train_gesture, [0, 4, 2, 3, 1]).reshape(-1, 32, 32, 2)
x_test_gesture = np.transpose(x_test_gesture, [0, 4, 2, 3, 1]).reshape(-1, 32, 32, 2)
x_val_gesture = np.transpose(x_val_gesture, [0, 4, 2, 3, 1]).reshape(-1, 32, 32, 2)


# 因為資料合併(14260, 35, 32, 32, 2) -> (499100, 32, 32, 2) 因此label需要重複35遍
gesture_labels_train = np.repeat(gesture_labels_train, 35)
gesture_labels_test = np.repeat(gesture_labels_test, 35)
gesture_labels_val = np.repeat(gesture_labels_val, 35)

# 先編碼再獨熱編碼
label_encoder = LabelEncoder()
gesture_labels_train_encoded = label_encoder.fit_transform(gesture_labels_train)
gesture_labels_test_encoded = label_encoder.transform(gesture_labels_test)
gesture_labels_val_encoded = label_encoder.transform(gesture_labels_val)

gesture_labels_train_one_hot = to_categorical(gesture_labels_train_encoded)
gesture_labels_test_one_hot = to_categorical(gesture_labels_test_encoded)
gesture_labels_val_one_hot = to_categorical(gesture_labels_val_encoded)

gesture_labels_train_one_hot = np.reshape(gesture_labels_train_one_hot, (-1, 1, 8))
gesture_labels_test_one_hot = np.reshape(gesture_labels_test_one_hot, (-1, 1, 8))
gesture_labels_val_one_hot = np.reshape(gesture_labels_val_one_hot, (-1, 1, 8))

print("Reshaped Gesture data train shape:", x_train_gesture.shape)
print("Reshaped Gesture labels train shape:", gesture_labels_train_one_hot.shape)

In [4]:
# 損失函數
def l2_loss(y_true, y_pred):
    diff = y_true - y_pred
    #print(f"Difference (diff): {diff.numpy()}")
    #print(f"y_true (label): {y_true}")
    #print(f"y_pred (label_pred): {y_pred}")
    loss = tf.reduce_mean(tf.square(diff))
    #print(f"loss: {loss}")
    return loss

In [4]:
model = load_model('CNN_TL_addDropout.h5')

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 2)]       0         
                                                                 
 online_cnn_fw/depthwise_con  (None, 15, 15, 2)        18        
 v2d_9/depthwise1 (Depthwise                                     
 Conv2D)                                                         
                                                                 
 online_cnn_fw/conv2d_9/Conv  (None, 15, 15, 32)       64        
 2D (Conv2D)                                                     
                                                                 
 online_cnn_fw/depthwise_con  (None, 7, 7, 32)         288       
 v2d_10/depthwise1 (Depthwis                                     
 eConv2D)                                                        
                                                             

In [6]:
# 加上全連接層
x = model.output

predictions = Dense(8, activation='softmax')(x)

model = Model(inputs=model.input, outputs=predictions)

In [ ]:
model.summary()

In [9]:
from keras.callbacks import ReduceLROnPlateau
learning_rate_function = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
# 訓練模型
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer, loss=l2_loss, metrics=['accuracy'])

history = model.fit(
    x_train_gesture, gesture_labels_train_one_hot,
    epochs=150,
    batch_size=4480,
    validation_data=(x_val_gesture, gesture_labels_val_one_hot),
    callbacks=[learning_rate_function]
)


In [12]:
model.save('ourCNNmodel_forTL_150_epoch_Dropout_0.7_withoutfalldown.h5')

In [ ]:
# 評估模型
test_loss, test_accuracy = model.evaluate(x_test_gesture, gesture_labels_test_one_hot)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# 畫圖
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'])

plt.show()